GloVe(Global Vectors for Word Representation)는 카운트 기반과 예측 기반을 모두 사용하는 방법론으로 2014년에 미국 스탠포드대학에서 개발한 단어 임베딩 방법론입니다. 앞서 있던 카운트 기반의 LSA(Latent Semantic Analysis)와 예측 기반의 Word2Vec의 단점을 보환하는 목적으로 나온 것입니다. 

현재까지 Word2Vec과 GloVe 중 성능이 더 좋다고 단정할 수 없습니다. 그렇기에 두 가지를 다 사용하고 성능이 더 좋은 것을 사용하는 것이 바람직합니다.

## **1. 기존 방법론에 대한 비판**

LSA는 각 단어의 빈도수를 카운트 한 행렬이라는 전체적인 통계 정보를 입력 받아 차원을 축소(Truncated SVD)하여 잠재된 의미를 끌어내는 방법론입니다. 반면, Word2Vec는 실제값과 예측값에 대한 오차를 손실 함수를 통해 줄여나가며 학습하는 예측 기반의 방법론입니다. 

LSA는 카운트 기반이기에 전체적인 통계 정보를 고려하기는 하지만, '왕:남자 = 여왕:?'과 같은 단어 의미의 유추 작업(Analogy task)에는 성능이 떨어집니다. Word2Vec는 예측 기반으로 단어 간 유추 작업에는 LSA보다 뛰어나지만, 임베딩 벡터가 윈도우 크기 내에서만 주변 단어를 고려하기 때문에 코퍼스의 전체적인 통계 정보를 반영하지 못합니다. GloVe는 이러한 기존 방법론들의 각각의 한계를 지적하며 LSA의 매커니즘이었던 카운트 기반의 방법과 Word2Vec의 매커니즘이었던 예측 기반의 방법론을 두 가지 모두 사용합니다. 

## **2. 원도우 기반 동시 등장 행렬(Window based Co-occurrence Matrix)**

단어의 동시 등장 행렬은 행과 열을 전체 단어 집합의 단어들을 구성하고, i 단어의 윈도우 크기(Window Size) 내에서 k 단어가 등장한 횟수를 i행 k열에 기재한 행렬을 말합니다. 예제를 보겠습니다.

Ex)  
I like deep learning  
I like NLP  
I enjoy flying  

윈도우 크기가 N일 때는 좌, 우에 존재하는 N개의 단어만 참고하게 됩니다. 윈도우 크기가 1일 때, 위 텍스트를 가지고 동시 등장 행렬은 다음과 같습니다.

|카운트|I|like|enjoy|deep|learning|NLP|flying|
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|I|0|2|1|0|0|0|0|
|like|2|0|0|1|0|1|0|
|enjoy|1|0|0|0|0|0|1|
|deep|0|1|0|0|1|0|0|
|learning|0|0|0|1|0|0|0|
|NLP|0|1|0|0|0|0|0|
|flying|0|0|1|0|0|0|0|

위 행렬은 행렬을 전치(Transpose)해도 동일한 행렬이 된다는 특징이 있습니다. 그 이유는 i 단어의 윈도우 크기 내에서 k 단어가 등장한 빈도는 반대로 k 단어의 윈도우 크기 내에서 i 단어가 등장한 빈도와 동일하기 때문입니다. 

참고 자료: http://web.stanford.edu/class/cs224n/slides/cs224n-2019-lecture02-wordvecs2.pdf

## **3. 동시 등장 확률(Co-occurrence Probability)**

이제 동시 등장 확률에 대해서 알아보겠습니다. 아래의 표는 어떤 동시 등장 행렬을 가지고 정리한 동시 등장 확률(Co-occurrence Probability)을 보여줍니다. 여기서 이야기하는 동시 등장 확률 $P(k|i)$는 동시 등장 행렬로부터 특정 단어 i의 전체 등장 횟수를 카운트하고, 특정 단어 i가 등장했을 때 어떤 단어 k가 등장한 횟수를 카운트하여 계산한 조건부 확률입니다. 

$P(k|i)$에서 i를 중심 단어(Center word), k를 주변 단어(Context word)라고 했을 때, 위에서 배운 동시 등장 행렬에서 중심 단어 i의 행의 모든 값을 더한 값을 분모로 하고 i행 k열의 값을 분자로 한 값이라고 볼 수 있겠습니다. 다음은 GloVe의 제안 논문에서 가져온 동시 등장 확률을 표로 정리한 하나의 예입니다.

|동시 등장 확률과 크기 관계 비(ratio)|k=solid|k=gas|k=water|k=fasion|
|:---:|:---:|:---:|:---:|:---:|
|$P(k \vert ice)$|0.00019|0.000066|0.003|0.000017|
|$P(k \vert steam)$|0.000022|0.00078|0.0022|0.000018|
|$\frac{P(k \vert ice)}{P(k \vert steam)}$|8.9|0.085|1.36|0.96|

위의 표를 통해 알 수 있는 사실은 solid가 등장했을 때, ice가 등장할 확률은 0.00019은 solid가 등장했을 때 steam이 등장할 확률인 0.000022보다 약 8.9배 크다는 사실입니다. 그도 그럴 것이 solid는 '단단한'이라는 의미를 가졌으니까 '증기'라는 의미를 가지는 steam보다는 당연히 '얼음'이라는 의미를 가지는 ice라는 단어와 더 자주 등장할 겁니다.

수식적으로 다시 정리하면 k가 solid일 때, $\frac{P(solid \vert ice)}{P(solid \vert steam)}$를 계산한 값은 8.9가 나옵니다. 이는 1보다 매우 큰 값입니다. 왜냐면 $P(solid \vert ice)$의 값은 크고, $P(solid \vert steam)$의 값은 작기 때문입니다.

그런데 k를 solid가 아니라 gas로 바꾸면 이야기는 완전히 달라집니다. gas는 ice보다는 steam과 더 자주 등장하므로, $\frac{P(gas \vert ice)}{P(gas \vert steam)}$를 계산한 값은 1보다 훨씬 작은 값인 0.085가 나옵니다. 반면, k가 water인 경우에는 solid와 steam 두 단어 모두와 동시 등장하는 경우가 많으므로 1에 가까운 값이 나오고, k가 fasion인 경우에는 solid와 steam 두 단어 모두와 동시 등장하는 경우가 적으므로 1에 가까운 값이 나옵니다. 보기 쉽도록 조금 단순화해서 표현한 표는 다음과 같습니다.

|동시 등장 확률과 크기 관계 비(ratio)|k=solid|k=gas|k=water|k=fasion|
|:---:|:---:|:---:|:---:|:---:|
|$P(k \vert ice)$|큰 값|작은 값|큰 값|작은 값|
|$P(k \vert steam)$|작은 값|큰 값|큰 값|작은 값|
|$\frac{P(k \vert ice)}{P(k \vert steam)}$|큰 값|작은 값|1에 가까움|1에 가까움|

## **4. 손실 함수(Loss function)**

우선 손실 함수를 설명하기 전에 각 용어를 정리하겠습니다.

- $X$: 동시 등장 행렬(Co-occurrence Matrix)
- $X_{ij}$: 중심 단어 i가 등장했을 때, 윈도우 내 주변 단어 j가 등장하는 횟수
- $X_i$: $\sum_j X_{ij}: 동시 등장 행렬에서 i행의 값을 모두 더한 값
- $P_{ik}$: $P(k \vert i) = \frac{X_{ik}}{X_i}: 중심 단어 i가 등장했을 때, 윈도우 내 주변 단어 k가 등장할 확률  
    Ex) $P(solid \vert ice)$ = 단어 ice가 등장했을 때, 단어 solid가 등장할 확률
- $\frac{P_{ik}}{P_{jk}}$: $P_{ik}$를 $P_{jk}$로 나눠준 값  
    Ex) $\frac{P(solid \vert ice)}{P(solid \vert steam)}$ = 8.9
- $w_i$: 중심 단어 i의 임베딩 벡터
- $\bar{w_k}: 주변 단어 k의 임베딩 벡터


GloVe의 아이디어를 한 줄로 요약하면 **'임베딩 된 중심 단어와 주변 단어 벡터의 내적이 전체 코퍼스에서의 동시 등장 확률이 되도록 만드는 것'** 입니다. 즉, 이를 만족하도록 임베딩 벡터를 만드는 것이 목표입니다. 이를 식으로 표현하면 다음과 같습니다.

$dot product(w_i, \bar{w_k}) \approx P(k \vert i) = P_{ik}$

뒤에서 보겠지만, 더 정확히는 GloVe는 아래와 같은 관계를 가지도록 임베딩 벡터를 설계합니다.

$dot product(w_i, \bar{w_k}) \approx log P(k \vert i) = log P_{ik}$

임베딩 벡터들을 만들기 위한 손실 함수를 처음부터 차근차근 설계해보겠습니다. 가장 중요한 것은 단어 간의 관계를 잘 표현하는 함수여야 한다는 것입니다. 이를 위해 앞서 배운 개념인 $P_{ik}/P_{jk}$를 식에 사용합니다. GloVe의 연구진들은 벡터 $w_i, w_j, \bar{w_k}$를 가지고 어떤 함수 $F$를 수행하면 $P_{ik}/P_{jk}$가 나온다는 초기 식으로부터 전개를 시작합니다.

$$F(w_i, w_j, \bar{w_k}) = \frac{P_{ik}}{P_{jk}}$$

아직 이 함수가 $F$가 어떤 식을 가지고 있는지는 정해진 것이 없습니다. 위의 목적에 맞게 근사할 수 있는 함수식은 무수히 많겠으나 최적의 식에 다가가기 위해서 단계별로 디테일을 추가하겠습니다. 함수 $F$는 두 단어 사이의 동시 등장 확률의 크기 관계 비(ratio) 정보를 벡터 공간에 인코딩하는 것이 목적입니다. 이를 위해 GloVe 연구진들은 $w_i$와 $w_j$라는 두 벡터의 차이를 함수 $F$의 입력으로 사용하는 것을 제안합니다.

$$F(w_i - w_j, \bar{w_k}) = \frac{P_{ik}}{P_{jk}}$$

그런데 우변은 스칼라값이고 좌변은 벡터값입니다. 이를 성립하기 위해서 함수 $F$의 두 입력에 내적(dot product)을 수행합니다.

$$F((w_i - w_j)^T \bar{w_k}) = \frac{P_{ik}}{P_{jk}}$$

정리하면, 선형 공간(Linear space)에서 단어의 의미 관계를 표현하기 위해 뺄셈과 내적을 택했습니다. 


여기서 함수 $F$가 만족해야 할 필수 조건이 있습니다. 중심 단어 $w$와 주변 단어 $\bar{w}$라는 선택 기준은 실제로는 무작위 선택이므로 이 둘의 관계는 자유롭게 교환될 수 있도록 해야합니다. 이것이 성립되게 하기 위해서 GloVe 연구진은 함수 $F$가 실수의 덧셈과 양수의 곱셈에 대해서 **준동형(Homomorphism)** 을 만족하도록 합니다. 이를 쉽게 정리하면 $a$와 $b$에 대해서 함수 $F$가 $F(a + b)$가 $F(a)F(b)$와 같도록 만족시켜야 한다는 의미입니다.

식으로 나타내면 다음과 같습니다.

$$F(a + b) = F(a)F(b), \forall a, b \in \mathbb{R}$$

이제 이 준동형식을 현재 전개하던 GloVe 식에 적용할 수 있도록 조금씩 바꿔보겠습니다. 전개하던 GloVe 식에 따르면, 함수 $F$는 결과값으로 스칼라 값($\frac{P_{ik}}{P_{jk}}$)이 나와야 합니다. 준동형식에서 $a$와 $b$가 각각 벡터값이라면 함수 $F$의 결과값으로는 스칼라 값이 나올 수 없지만, $a$와 $b$가 각각 사실 두 벡터의 내적값이라고 하면 결과값으로 스칼라 값이 나올 수 있습니다. 그러므로 위의 준동형식을 아래와 같이 바꿔보겠습니다. 여기서 $v_1, v_2, v_3, v_4$는 각각 벡터값입니다. 아래의 $V$는 벡터를 의미합니다.

$$F(v_1^T v_2 + v_3^T v_4) = F(v_1^T v_2)F(v_3^T v_4), \forall v_1, v_2, v_3, v_4 \in V$$

그런데 앞서 작성한 GloVe 식에서는 $w_i$와 $w_j$라는 두 벡터의 차이를 함수 $F$의 입력으로 받았습니다. GloVe식에 바로 적용을 위해 준동형식을 뺄셈에 대한 준동형식으로 변경합니다. 그러면 곱셈도 나눗셈으로 바뀌게 됩니다.

$$F(v_1^T v_2 - v_3^T v_4) = \frac{F(v_1^T v_2)}{F(v_3^T v_4)}, \forall v_1, v_2, v_3, v_4 \in V$$

이제 이 준동형식을 GloVe 식에 적용하겠습니다. 우선, 함수 $F$의 우변은 다음과 같이 바뀌어야 합니다.

$$F((w_i - w_j)^T \bar{w_k}) = \frac{F(w_i^T \bar{w_k})}{F(w_j^T \bar{w_k})}$$

그런데 이전의 식에 따르면 우변은 본래 $\frac{P_{ik}}{P_{jk}}$였으므로, 결과적으로 다음과 같습니다.

$$\frac{P_{ik}}{P_{jk}} = \frac{F(w_i^T \bar{w_k})}{F(w_j^T \bar{w_k})}$$

$$F(w_i^T \bar{w_k}) = P_{ik} = \frac{X_{ik}}{X_i}$$

좌변을 풀어쓰면 다음과 같습니다.

$$F(w_i^T \bar{w_k} - w_j^T \bar{w_k}) = \frac{F(w_i^T \bar{w_k})}{F(w_j^T \bar{w_k})}$$

이는 뺄셈에 대한 준동형식의 형태와 정확히 일치합니다. 이제 이를 만족하는 함수 $F$를 찾아야 합니다. 그리고 이를 정확하게 만족시키는 함수가 있는데 바로 지수 함수입니다. $F$를 지수 함수 **exp** 라고 해봅시다.

$$exp(w_i^T \bar{w_k} - w_j^T \bar{w_k}) = \frac{exp(w_i^T \bar{w_k})}{exp(w_j^T \bar{w_k})}$$

$$exo(w_i^T \bar{w_k}) = P_{ik} = \frac{X_{ik}}{X_i}$$

위의 두 번째 식으로부터 다음과 같은 식을 얻을 수 있습니다.

$$w_i^T \bar{w_k} = log P_{ik} = log(\frac{X_{ik}}{X_i}) = log X_{ik} - log X_i$$

그런데 여기서 상기해야할 것은 앞서 언급했듯이, 사실 $w_i$와 $\bar{w_k}$는 두 값의 위치를 서로 바꾸어도 식이 성립해야 합니다. $X_{ik}$의 정의를 생각해보면 $X_{ki}$와도 같습니다. 그런데 이게 성립되려면 위의 식에서 $log X_i$ 항이 걸림돌입니다. 이 부분만 없다면 이를 성립시킬 수 있습니다. 그래서 GloVe 연구팀은 이 $log X_i$항을 $w_i$에 대한 편향 $b_i$라는 상수항으로 대체하기로 합니다. 같은 이유로 $\bar{w_k}$에 대한 편향 $\bar{b_k}$를 추가합니다.

$$w_i^T \bar{w_k} + b_i + \bar{b_k} = log X_{ik}$$

이 식이 손실 함수의 핵심이 되는 식입니다. 우변의 값과 차이를 최소화하는 방향으로 좌변의 4개의 항이 학습을 통해 바뀌게 됩니다. 즉, 손실 함수는 다음과 같이 일반화될 수 있습니다.

$$Loss function = \sum_{m, n = 1}^V (w_m^T \bar{w_n} + b_m + \bar{b_n} - log X_{mn})^2$$

여기서 $V$는 단어 집합의 크기를 의미합니다. 그런데 아직 최적의 손실 함수라기에는 부족합니다. GloVe 연구진은 $log X_{ik}$에서 $X_{ik}$값이 0이 될 수 있음을 지적합니다. 대안 중 하나는 $log X_{ik}$항을 $log (1 + X_{ik})$로 변경하는 것입니다. 하지만 이렇게 해도 여전히 해결되지 않는 문제가 있습니다.

바로 동시 등장 행렬 $X$는 마치 DTM처럼 희소 행렬일 가능성이 다분하다는 점입니다. 동시 등장 행렬 $X$에는 많은 값이 0이거나, 동시 등장 빈도가 적어서 많은 값이 작은 수치를 가지는 경우가 많습니다. 앞서 빈도수를 가지고 가중치를 주는 고민을 하는 TF-ID나 LSA와 같은 몇 가지 방법들을 본 적이 있습니다. GloVe의 연구진은 동시 등장 행렬에서 동시 등장 빈도의 값 $X_{ik}$이 굉장히 낮은 경우에는 정보에 거의 도움이 되지 않는다고 판단합니다. 그래서 이에 대한 가중치를 주는 고민을 하게 되는데 GloVe 연구팀이 선택한 것은 바로 $X_{ik}$의 값에 영향을 받는 가중치 함수 $f(X_{ik})$를 손실 함수에 도입하는 것입니다. 

GloVe에 도입되는 $f(X_{ik})$의 그래프를 그려보겠습니다.

<img src = "https://wikidocs.net/images/page/22885/%EA%B0%80%EC%A4%91%EC%B9%98.PNG">

$X_{ik}$의 값이 작으면 상대적으로 함수의 값은 작도록 하고, 값이 크면 함수의 값은 상대적으로 크도록 합니다. 하지만 $X_{ik}$가 지나치게 높다고해서 지나친 가중치를 주지 않기 위해 함수의 최대값은 정해져 있습니다. (최대값은 1) 예를 들어 'It is'와 같은 불용어의 동시 등장 빈도수가 높다고해서 지나친 가중을 받아서는 안 됩니다. 이 함수의 값을 손실 함수에 곱해주면 가중치의 역할을 할 수 있습니다.

이 함수 $f(x)$의 식은 다음과 같이 정의됩니다. 

$$f(x) = min(1, (\frac{x}{x_{max}}^{\frac{3}{4}}))$$

최종적으로 다음과 같은 일반화 된 손실 함수를 얻어낼 수 있습니다.

$$Loss function = \sum_{m, n = 1}^V f(X_{mn})(w_m^T \bar{w_n} + b_m + \bar{b_n} - log X_{mn})^2$$

## **5. GloVe 훈련시키기**

In [5]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

In [6]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x19fedaacac0>)

훈련 데이터 파일은 xml 문법으로 작성되어 있어 자연어를 얻기 위해서 전처리가 필요합니다. 얻고자 하는 실질적 데이터는 영어 문장으로만 구성된 내용을 담고 있는 <content>와 </content> 사이의 내용입니다. 전처리 작업을 통해 xml 문법들은 제거하고, 해당 데이터만 가져와야 합니다. 뿐만 아리나, <content>와 </content> 사이의 내용 중에는 (Laughter)나 (Applause)같은 배경음을 나타내는 단어도 제거해야 합니다.

In [12]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거
# 해당 코드는 괄호로 구성된 내용을 제거
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환
normalized_text = []
for string in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행
result = [word_tokenize(sentence) for sentence in normalized_text]

In [13]:
from glove import Corpus, Glove

# 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬 생성
corpus = Corpus()
corpus.fit(result, window=5)

# 학습에 이용할 쓰레드의 개수는 4로 설정, 에포크는 20
glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


이제 학습이 완료되었습니다. glove.most_similar()는 입력 단어와 가장 유사한 단어들의 리스트를 리턴합니다. 

In [14]:
model_result1=glove.most_similar("man")
print(model_result1)

[('woman', 0.9589456888977812), ('guy', 0.887088817037771), ('girl', 0.8807891002587276), ('kid', 0.8438778113165954)]


In [15]:
model_result2=glove.most_similar("boy")
print(model_result2)

[('girl', 0.9340611748024038), ('man', 0.8359945874344212), ('kid', 0.8343818890309035), ('woman', 0.8328161784801258)]


In [16]:
model_result3=glove.most_similar("university")
print(model_result3)

[('harvard', 0.8872641740819832), ('mit', 0.8574851821799727), ('stanford', 0.8383140421855932), ('cambridge', 0.8295228402624245)]


In [17]:
model_result4=glove.most_similar("water")
print(model_result4)

[('air', 0.8405225323193627), ('clean', 0.8377991373768472), ('fresh', 0.8318094316003998), ('electricity', 0.8116933834970668)]


In [18]:
model_result5=glove.most_similar("physics")
print(model_result5)

[('beauty', 0.8802554586227059), ('economics', 0.878198386046823), ('chemistry', 0.874188979656539), ('mathematics', 0.8597018278252551)]


In [19]:
model_result6=glove.most_similar("muscle")
print(model_result6)

[('tissue', 0.8383606179647394), ('nerve', 0.8367805654544072), ('skeletal', 0.7914129214776572), ('stem', 0.7694722939819497)]


In [20]:
model_result7=glove.most_similar("clean")
print(model_result7)

[('fresh', 0.8421838781527502), ('water', 0.8377991373768472), ('heat', 0.8059021787205549), ('air', 0.7961272118764987)]
